In [154]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline  

import datetime

In [3]:
# import data from local files
global_mobility = pd.read_csv('../../data/raw_data/google_mobility.csv', header=0, names=[
    'country_code', 'country', 'region_1', 'region_2', 'date', 'retail_recreation', \
    'grocery_pharmacy', 'parks', 'transit', 'workplaces', 'residential'
], low_memory=False)
covid19_canada = pd.read_csv('../../data/raw_data/covid19_canada.csv', low_memory=False)
covid19_global = pd.read_csv('../../data/raw_data/covid19_global.csv', low_memory=False)
weather_ontario = pd.read_csv('../../data/raw_data/weather_ontario.csv', low_memory=False)

# import data from URLs
# global_mobility_url = pd.read_csv('https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=6d352e35dcffafce')
# covid19_canada_url = pd.read_csv('https://health-infobase.canada.ca/src/data/covidLive/covid19.csv')

In [4]:
global_mobility.head()

,country_code,country,region_1,region_2,date,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential
0,AE,United Arab Emirates,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [5]:
# pd.set_option('display.max_rows', canada_provinces_mobility.shape[0]+1)c
# pd.set_option('display.float_format', lambda x: '%.f' % x)

# Create data for Canada provinces

In [6]:
# create dataframe for Canada 
canada_mobility = global_mobility.loc[global_mobility.country_code == 'CA']

# drop 'region_2' because all values are NaN
canada_mobility = canada_mobility.drop('region_2', 1)

# export data 
canada_mobility.to_csv(path_or_buf='../../data/cleaned_data/canada_mobility.csv')

In [7]:
canada_mobility.head()

,country_code,country,region_1,date,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential
16799,CA,Canada,NaN,2020-02-15,4.0,2.0,10.0,3.0,1.0,0.0
16800,CA,Canada,NaN,2020-02-16,13.0,8.0,41.0,4.0,0.0,-2.0
16801,CA,Canada,NaN,2020-02-17,-12.0,-15.0,63.0,-28.0,-52.0,11.0
16802,CA,Canada,NaN,2020-02-18,-1.0,4.0,6.0,-1.0,-1.0,1.0
16803,CA,Canada,NaN,2020-02-19,1.0,1.0,9.0,0.0,0.0,0.0


In [8]:
# create dataframe for provinces in Canada

# drop 'Prince Edward Island' since there are no many missing values
non_provinces = [np.NaN, 'Northwest Territories', 'Nunavut', 'Yukon', 'Prince Edward Island']
non_provinces_bool = ~canada_mobility.region_1.isin(non_provinces) 
canada_provinces_mobility = canada_mobility[non_provinces_bool]

# drop rows that contain null values in 'parks' and 'transit' columns since they occupy less than 1% of the records
# filling null values with mean/median may not be a good idea 
# since we will be using insights from future data which will not be available at the time
canada_provinces_mobility.parks.isna().sum() / len(canada_provinces_mobility.parks)
canada_provinces_mobility.transit.isna().sum() / len(canada_provinces_mobility.transit)
canada_provinces_mobility = canada_provinces_mobility.dropna(subset=['parks', 'transit'])

# export data
canada_provinces_mobility.to_csv(path_or_buf='../../data/cleaned_data/canada_provinces_mobility.csv')

In [9]:
canada_provinces_mobility.tail()

,country_code,country,region_1,date,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential
18094,CA,Canada,Saskatchewan,2020-05-21,-12.0,20.0,80.0,-38.0,-39.0,13.0
18095,CA,Canada,Saskatchewan,2020-05-22,-21.0,8.0,83.0,-36.0,-37.0,12.0
18096,CA,Canada,Saskatchewan,2020-05-23,-24.0,6.0,68.0,-27.0,-14.0,6.0
18097,CA,Canada,Saskatchewan,2020-05-24,-19.0,3.0,197.0,-24.0,-16.0,3.0
18098,CA,Canada,Saskatchewan,2020-05-25,-13.0,11.0,151.0,-38.0,-39.0,10.0


# Create Google trends park data and Canada provinces park data

In [10]:
# import data
google_trends_park = pd.read_csv('../../data/raw_data/google_trends_park.csv')

# convert date to datetime
google_trends_park.date = pd.to_datetime(google_trends_park.date, infer_datetime_format=True)
google_trends_park = google_trends_park[google_trends_park.date <= '2020-05-25']

# export data
google_trends_park.to_csv(path_or_buf='../../data/cleaned_data/google_trends_park.csv')

Since Google trends does not provide daily data for the past 12 months and provides daily data for the past 90 days, we used the data of Google search trends for "park" for the past 90 days.

In [11]:
# create dataframe for Canada province park data

# drop unused columns
canada_provinces_park  = canada_provinces_mobility.drop(['country_code', 'country', 'retail_recreation', \
                                                         'grocery_pharmacy', 'transit', 'workplaces', 
                                                         'residential'], axis=1)

# group data by date
canada_provinces_park = canada_provinces_park.groupby('date').mean().reset_index()

# drop date before 2020-03-04 since Google trends data does not contain data before this date
canada_provinces_park = canada_provinces_park[canada_provinces_park.date >= '2020-03-04']

# export data
canada_provinces_park.to_csv(path_or_buf='../../data/cleaned_data/canada_provinces_park.csv')

# Standardize data for each category, region_1, date & Create social-distancing score for each date

In [12]:
# import data
canada_provinces_mobility = pd.read_csv('../../data/cleaned_data/canada_provinces_mobility.csv')

canada_provinces_mobility.head()

,Unnamed: 0,country_code,country,region_1,date,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential
0,16900,CA,Canada,Alberta,2020-02-15,7.0,3.0,24.0,7.0,-2.0,-1.0
1,16901,CA,Canada,Alberta,2020-02-16,10.0,-2.0,28.0,3.0,-3.0,-2.0
2,16902,CA,Canada,Alberta,2020-02-17,-7.0,-2.0,10.0,-40.0,-67.0,15.0
3,16903,CA,Canada,Alberta,2020-02-18,-1.0,1.0,-2.0,-9.0,-5.0,2.0
4,16904,CA,Canada,Alberta,2020-02-19,3.0,4.0,21.0,-7.0,-1.0,1.0


In [13]:
# Calculate mean and standard deviation for mobility score by provinces
mean_mobility_provinces = canada_provinces_mobility.groupby('region_1').mean().reset_index()
std_mobility_provinces = canada_provinces_mobility.groupby('region_1').std().reset_index()

In [14]:
# z-score
def standardize_data_z(province, category, i):
    canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] = \
    (canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] - \
    mean_mobility_provinces.loc[mean_mobility_provinces.region_1 == province, category][i]).\
    div(std_mobility_provinces.loc[std_mobility_provinces.region_1 == province, category][i]) \


def standardize_data_min_max(province, category):
#     min-max normalization between [-100, 100]
#     canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] = \
#     (200 * (canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] - \
#     min(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category])) /\
#     (max(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category]) - \
#     min(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category])) - 100)
#     min-max normalization between [0, 100]
    canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] = \
    100 * ((canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category] - \
    min(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category])) / \
    (max(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category]) - \
    min(canada_provinces_mobility.loc[canada_provinces_mobility.region_1 == province, category])))

In [15]:
std_mobility_provinces.loc[std_mobility_provinces.region_1 == 'Alberta', 'retail_recreation'][0]

21.427732562552603

In [16]:
provinces = ['Alberta', 'British Columbia', 'Manitoba', 'New Brunswick', 'Newfoundland and Labrador', \
             'Nova Scotia', 'Ontario', 'Quebec', 'Saskatchewan']
categories = ['retail_recreation', 'grocery_pharmacy', 'parks', 'transit', 'workplaces', 'residential']

In [17]:
# standardize data by z-score
i = 0
for province in provinces:
    for category in categories:
        standardize_data_z(province, category, i)
    i += 1

In [18]:
# create new feature Social Distancing Score for each row

# canada_provinces_mobility['s_d_score'] = -1 * (canada_provinces_mobility['parks']) + \
#                                                canada_provinces_mobility.iloc[:, -6:-5].sum(axis=1) + \
#                                                canada_provinces_mobility.iloc[:, -3:-2].sum(axis=1) + \
#                                                canada_provinces_mobility['residential']

# canada_provinces_mobility['s_d_score'] = canada_provinces_mobility.iloc[:, -6:-1].sum(axis=1)

canada_provinces_mobility['s_d_score'] = -1 * (canada_provinces_mobility.iloc[:, -6:-2].sum(axis=1)) + \
canada_provinces_mobility['residential']


In [19]:
# standardize data by min_max
i = 0
for province in provinces:
    for category in categories:
        standardize_data_min_max(province, 's_d_score')
        standardize_data_min_max(province, category)
    i += 1

In [20]:
canada_provinces_mobility = canada_provinces_mobility.drop(['country_code', 'country', 'Unnamed: 0'], axis=1)

# export data
canada_provinces_mobility.to_csv(path_or_buf='../../data/cleaned_data/standardized_mobility_data.csv')

# Create social distancing score for each province

In [21]:
# import data
mobility_data = pd.read_csv('../../data/cleaned_data/standardized_mobility_data.csv', parse_dates=['date'], \
                           index_col='Unnamed: 0')

# drop column
# mobility_data = mobility_data.drop(['Unnamed: 0.1'], axis=1)

In [22]:
mobility_data.columns

Index(['region_1', 'date', 'retail_recreation', 'grocery_pharmacy', 'parks',
       'transit', 'workplaces', 'residential', 's_d_score'],
      dtype='object')

In [23]:
mobility_data = mobility_data.groupby('region_1').mean().reset_index()
mobility_data.sort_values(by=['s_d_score'], axis=0, ascending=False)

,region_1,retail_recreation,grocery_pharmacy,parks,transit,workplaces,residential,s_d_score
5,Nova Scotia,40.390071,46.330935,19.816842,36.567797,47.271845,46.850000,54.441147
7,Quebec,48.888664,57.434910,33.172698,36.833790,49.097263,48.599717,51.190799
0,Alberta,47.741760,51.066797,26.170456,39.462518,54.832626,43.011066,48.758080
6,Ontario,48.262275,52.184576,31.873808,39.227292,50.547160,44.267848,47.899812
1,British Columbia,42.505501,41.526761,36.227812,35.939308,52.558750,48.958689,47.621814
8,Saskatchewan,49.504950,52.809567,30.488503,44.554455,58.210577,40.990099,44.244466
3,New Brunswick,53.307749,62.801791,30.116959,48.503236,53.611866,39.903846,42.924744
4,Newfoundland and Labrador,52.400389,53.128689,26.324238,43.799058,41.094620,41.296519,39.834350
2,Manitoba,58.290657,57.102056,27.572524,40.617633,60.373790,37.991513,38.410132


In [24]:
# round data to integers
for category in categories:
    mobility_data[category] = mobility_data[category].astype(int)
mobility_data['s_d_score'] = mobility_data['s_d_score'].astype(int)
    
# Rename columns
mobility_data = mobility_data.rename(columns={'region_1': 'Province', 's_d_score': 'Social Distancing Score', \
                             'retail_recreation': 'Retail & Recreation', \
                             'grocery_pharmacy': 'Grocery & Pharmacy', 'parks': 'Parks', \
                             'transit': 'Transit', 'workplaces': 'Workplaces',
                             'residential': 'Residential'})

# export data
mobility_data.to_csv(path_or_buf='../../data/cleaned_data/social_distancing_score_province.csv')

In [25]:
mobility_data

,Province,Retail & Recreation,Grocery & Pharmacy,Parks,Transit,Workplaces,Residential,Social Distancing Score
0,Alberta,47,51,26,39,54,43,48
1,British Columbia,42,41,36,35,52,48,47
2,Manitoba,58,57,27,40,60,37,38
3,New Brunswick,53,62,30,48,53,39,42
4,Newfoundland and Labrador,52,53,26,43,41,41,39
5,Nova Scotia,40,46,19,36,47,46,54
6,Ontario,48,52,31,39,50,44,47
7,Quebec,48,57,33,36,49,48,51
8,Saskatchewan,49,52,30,44,58,40,44


# Create dataset for modeling

In [30]:
# import data
modeling_data = pd.read_csv('../../data/cleaned_data/standardized_mobility_data.csv', parse_dates=['date'])

# drop unnecessary columns
modeling_data.drop(['Unnamed: 0', 'retail_recreation', 'grocery_pharmacy', 'parks', 'transit', 'workplaces', \
                    'residential'], axis=1, inplace=True)

In [143]:
# create data for the daily increase in the number of cases in Canada

# import data for the number cases in Canada
covid19_canada = pd.read_csv('../../data/raw_data/covid19_canada.csv')

# format date
covid19_canada.date = pd.to_datetime(covid19_canada.date, format='%d-%m-%Y')

# filter rows
covid19_canada = covid19_canada.loc[covid19_canada.pruid == 1]
covid19_canada = covid19_canada.loc[covid19_canada.date >= '2020-02-05']

# select columns
covid19_canada = covid19_canada[['date', 'numconf']]

# create new feature - number of increase compare to the day before
covid19_canada['New cases'] = covid19_canada.numconf - covid19_canada.numconf.shift(1, fill_value = 7)

# rename columns
covid19_canada = covid19_canada.rename(columns={'date': 'Date', 'numconf': 'Confirmed cases'})

In [144]:
covid19_canada

,Date,Confirmed cases,New cases
5,2020-02-08,7,0
8,2020-02-16,8,1
11,2020-02-21,9,1
14,2020-02-24,10,1
17,2020-02-25,11,1
20,2020-02-26,12,1
23,2020-02-27,13,1
26,2020-02-29,15,2
30,2020-03-01,24,9
34,2020-03-03,33,9


In [166]:
# create data for the daily increase in the number of cases around the world

# import data
covid19_globe = pd.read_csv('../../data/raw_data/covid19_global.csv')

# format date
covid19_globe.Date_reported = covid19_globe.Date_reported.str.slice(stop=10)

# rename columns
covid19_globe = covid19_globe.rename(columns={'Date_reported': 'Date', 'New_cases': 'New cases', \
                                              'New_deaths': 'New deaths', 'Cumulative_cases': 'Cumulative cases', \
                                              'Cumulative_deaths': 'Cumulative deaths'})

# group by date
covid19_globe = covid19_globe.groupby('Date').sum().reset_index()

# filter rows
covid19_globe = covid19_globe.loc[covid19_globe.Date >= '2020-02-15']

In [167]:
covid19_globe

,Date,New cases,Cumulative cases,New deaths,Cumulative deaths
35,2020-02-15,2732,67187,144,1527
36,2020-02-16,2090,69277,142,1669
37,2020-02-17,2161,71438,106,1775
38,2020-02-18,1993,73431,98,1873
39,2020-02-19,1856,75287,138,2011
40,2020-02-20,486,75773,118,2129
41,2020-02-21,1044,76817,120,2249
42,2020-02-22,1109,77926,113,2362
43,2020-02-23,1020,78946,101,2463
44,2020-02-24,619,79565,164,2627


In [ ]:
# clean data for daily temperature and precipitation in Canada

# import data
